<a href="https://colab.research.google.com/github/romanzes637/algebra/blob/master/algebra3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
tf.debugging.set_log_device_placement(True)

TensorFlow 2.x selected.


In [0]:
import numpy as np
import functools 


# with tf.device('/GPU:0'):
# with tf.device('/CPU:0'):
np.random.seed(0)
tf.random.set_seed(0)
n = 20000
m = 100
x1 = []
x2 = []
y = []
for j in range(n):
   a = np.random.randint(-m, m)
   b = np.random.randint(-m, m)
  #  c = a*b
  #  c = a + b
   dx1 = np.random.randint(0, m)
   dx2 = np.random.randint(0, m)
   dy = np.random.randint(0, m)
  #  x1.append([a, 0]) if a >= 0 else x1.append([0, abs(a)])
  #  x2.append([b, 0]) if b >= 0 else x2.append([0, abs(b)])
  #  y.append([c, 0]) if c >= 0 else y.append([0, abs(c)])
   x1.append([a+dx1, dx1]) if a >= 0 else x1.append([dx1, abs(a)+dx1])
   x2.append([b+dx2, dx2]) if b >= 0 else x2.append([dx2, abs(b)+dx2])
  #  y.append([x1[-1][0]*x2[-1][0] + x1[-1][1]*x2[-1][1], 
  #            x1[-1][0]*x2[-1][1] + x1[-1][1]*x2[-1][0]]) 
   y.append([x1[-1][0] + x2[-1][0], x1[-1][1] + x2[-1][1]])  # - + - = -, + + + = +
  #  y.append([x1[-1][1] + x2[-1][1], x1[-1][0] + x2[-1][0]])  # - + - = +, + + + = -
  #  y.append([c+dy, dy]) if c >= 0 else y.append([dy, abs(c)+dy])
print(x1[0], x2[0], y[0])

# w = tf.random.uniform((16, 2), minval=0, maxval=2, dtype=tf.int32)
w = tf.ones((16, 2), dtype=tf.int32)
# w = tf.constant([
#                 [0, 0],
#                 [0, 0],
#                 [0, 0],
#                 [0, 0],
#                 [0, 0],
#                 [1, 0],
#                 [0, 1],
#                 [0, 0],
#                 [0, 0],
#                 [0, 1],
#                 [1, 0],
#                 [0, 0],
#                 [0, 0],
#                 [0, 0],
#                 [0, 0],
#                 [0, 0]])
# w = tf.constant([
#                  [0, 0],
#                  [1, 0],
#                  [0, 1],
#                  [0, 0],
#                  [1, 0],
#                  [0, 0],
#                  [0, 0],
#                  [0, 0],
#                  [0, 1],
#                  [0, 0],
#                  [0, 0],
#                  [0, 0],
#                  [0, 0],
#                  [0, 0],
#                  [0, 0],
#                  [0, 0]
# ])
adw = tf.ones_like(w)
# for j in range(4):
for i in range(n):
  a = tf.constant(functools.reduce(lambda x, y: x + [z*y for z in x], x1[i] + x2[i], [1]))
  t = tf.constant(y[i])
  z = tf.einsum('ij,i->j', w, a)  # only on CPU, because GPU doesn't have int kernel
  # z = tf.linalg.matvec(w, a, transpose_a=True)
  s = t + tf.reverse(z, axis=[0])
  if tf.math.is_strictly_increasing(s):
    loss = tf.constant([0, 1])
  elif tf.math.is_non_decreasing(s):
    loss = tf.constant([0, 0])
  else:
    loss = tf.constant([1, 0])
  a = tf.clip_by_value(a, clip_value_min=0, clip_value_max=1)
  dw = tf.einsum('i,j->ij', a, loss)  # only on CPU, because GPU doesn't have int kernel
  adw += dw
  # w += dw
print(adw)
print(adw[:,0] - adw[:,1])
# print(w)

[136, 64] [67, 120] [203, 184]
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Einsum in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReverseV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op All in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LessEqual in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Minimum in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Maximum in device /job:localhost/replica:0/task:0/devi

In [0]:
# with tf.device('/GPU:0'):
# with tf.device('/CPU:0'):
a = tf.constant([0., 1.])
w = tf.Variable([[1., 1.],
                 [1., 1.]])
t = tf.constant([1., 0.])
for i in range(1000):
  with tf.GradientTape() as tape:
    z = tf.einsum('ij,j->i', w, a)
    loss = t - z
  dl_dw = tape.gradient(loss, w)
  dw = tf.einsum('ij,i->ij', dl_dw, loss)
  w.assign_sub(0.01*dw)
  if i % 250 == 0:
    print(dw)
    print(w)
    print(dt_dw)
    print(t)
    print(z)
    print(loss)
w = tf.constant([[0., 0.],
                 [1., 0.],
                 [0., 1.]])

a = tf.constant([1., 2., 3.])
print(tf.einsum('ij,i->j', w, a))
print(tf.einsum('ij,j->i', tf.transpose(w), a))

# powerset

In [41]:
import functools
import numpy as np
import time
from itertools import chain, combinations, accumulate
import operator

n = 20  # n = 27 - limit, 14 - errors in tensorflow

print('python + numpy')
t0 = time.time()
s = np.arange(0, n, 1)
m0 = np.array([[x // 2**y % 2 for y in range(n)] for x in range(2**n)])
# m1 = np.array([[(x // 2**y + 1) % 2 for y in range(n)] for x in range(2**n)])
# m1 = np.logical_not(m0).astype(np.int)
m1 = np.ones_like(m0) - m0
ps = np.einsum('ij,j->ij', m0, s)
ps1 = ps + m1
pps = np.prod(ps1, axis=1)
print(time.time() - t0)
print(pps[:4], pps[-4:], len(pps), pps.dtype)

print('python')
t0 = time.time()
s = [x for x in range(n)]
pps = functools.reduce(lambda x, y: x + [z*y for z in x], s, [1])
print(time.time() - t0)
print(pps[:4], pps[-4:], len(pps), type(pps[0]))

print('python itertools, different order!')
t0 = time.time()
s = [x for x in range(n)]
pps = [1] + [list(accumulate(c, operator.mul))[-1] 
                  for r in range(1, len(s) + 1) 
                  for c in combinations(s, r)]  # TODO add initial [1] py 3.8+
print(time.time() - t0)
print(pps[:4], pps[-4:], len(pps), type(pps[0]))

print('python + tensorflow')
t0 = time.time()
s = tf.constant([x for x in range(n)])
m0 = tf.constant([[x // 2**y % 2 for y in range(n)] for x in range(2**n)])
# m1 = tf.constant([[(x // 2**y + 1) % 2 for y in range(n)] for x in range(2**n)])
# m1 = tf.logical_not(m0).dtype(tf.int32)
m1 = tf.ones_like(m0) - m0
ps = tf.einsum('ij,j->ij', m0, s)
ps1 = ps + m1
pps = tf.reduce_prod(ps1, axis=1)
print(time.time() - t0)
print(pps)

python + numpy
10.014102935791016
[1 0 1 0] [121645100408832000                  0 121645100408832000
                  0] 1048576 int64
python
0.07720327377319336
[1, 0, 1, 0] [121645100408832000, 0, 121645100408832000, 0] 1048576 <class 'int'>
python itertools, different order!
1.2571401596069336
[1, 0, 1, 2] [0, 0, 121645100408832000, 0] 1048576 <class 'int'>
python + tensorflow


NameError: ignored

## powerset layer

In [17]:
import time
import functools  # https://docs.python.org/3/library/functools.html
import operator  # see https://docs.python.org/3/library/operator.html
import itertools  # see https://docs.python.org/3/library/itertools.html

n = 12  # n > 25 works only with itertools, n < 12 works fast
s = [x for x in range(n)]
min_len = 0
max_len = n
lens = [x for x in range(min_len, max_len + 1)]
# lens = [0, 1, 2]

if n <= 12:
  print('python')
  t0 = time.time()
  # max and min len
  # ps = list(filter(lambda x: min_len <= len(x) <= max_len, 
  #                  functools.reduce(lambda x, y: x + [z + [y] for z in x], s, [[]])))
  # len in lens
  ps = list(filter(lambda x: len(x) in lens,
                  functools.reduce(lambda x, y: x + [z + [y] for z in x], s, [[]])))
  # mul
  ps_mul = [functools.reduce(operator.mul, z, 1) for z in ps]
  # pow and add, set 1 to empty set (bias)
  if min_len == 0 or 0 in lens:
    ps_pow = [1] + [functools.reduce(operator.pow, z) for z in ps[1:]]
    ps_add = [1] + [functools.reduce(operator.add, z) for z in ps[1:]]
  else:
    ps_pow = [functools.reduce(operator.pow, z) for z in ps]
    ps_add = [functools.reduce(operator.add, z) for z in ps]
  print(time.time() - t0)
  # print(ps)
  # print(ps_mul)
  # print(ps_add)
  # print(ps_pow)
  print(len(ps))
  print(len(ps_mul))
  print(len(ps_add))
  print(len(ps_pow))

print('python itertools')
t0 = time.time()
# powerset
ps = list(itertools.chain.from_iterable(itertools.combinations(s, x) for x in lens))
# mul
ps_mul = list(functools.reduce(operator.mul, y, 1) for x in lens for y in itertools.combinations(s, x))
# add
ps_add = list(functools.reduce(operator.add, y, 0) for x in lens for y in itertools.combinations(s, x))
if 0 in lens:  # empty set (bias)
  ps_add[0] = 1
# pow
if 0 in lens:  # because pow operator with initial value 1 returns 1 and 0 returns 0
  lens.remove(0)
ps_pow = list(functools.reduce(operator.pow, y) for x in lens for y in itertools.combinations(s, x))
ps_pow = [1] + ps_pow
print(time.time() - t0)
# print(ps)
# print(ps_mul)
# print(ps_add)
# print(ps_pow)
print(len(ps))
print(len(ps_mul))
print(len(ps_add))
print(len(ps_pow))

python
4.455255508422852
4096
4096
4096
4096
python itertools
4.34265398979187
4096
4096
4096
4096
